In [80]:
from logger import setup_logger
from Face_parsing_model import BiSeNet

import torch

import os
import os.path as osp
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import cv2

In [81]:
def vis_parsing_maps(im, parsing_anno, stride, save_im=False, save_path='vis_results/parsing_map_on_im.jpg'):
    # Colors for all 20 parts
    part_colors = [[0, 0, 0]] * 19  # Default black for all parts
    part_colors[1] = [255, 255, 255]  # Skin - white
    part_colors[10] = [255, 255, 255]  # Nose - white

    # 입력 이미지를 RGB -> BGR로 변환
    im = np.array(im)

    vis_im = im.copy().astype(np.uint8)  # 원본 이미지 복사
    vis_parsing_anno = parsing_anno.copy().astype(np.uint8)  # 마스크 복사
    vis_parsing_anno = cv2.resize(vis_parsing_anno, None, fx=stride, fy=stride, interpolation=cv2.INTER_NEAREST)

    # 배경을 검정색으로 채운 컬러 마스크 생성
    vis_parsing_anno_color = np.zeros((vis_parsing_anno.shape[0], vis_parsing_anno.shape[1], 3), dtype=np.uint8)

    # 각 파트에 맞는 색을 채우기
    num_of_class = np.max(vis_parsing_anno)
    for pi in range(1, num_of_class + 1):
        index = np.where(vis_parsing_anno == pi)
        vis_parsing_anno_color[index[0], index[1], :] = part_colors[pi]

    # 마스크와 원본 이미지의 채널 맞추기
    if vis_parsing_anno_color.shape[:2] != vis_im.shape[:2]:
        vis_parsing_anno_color = cv2.resize(vis_parsing_anno_color, (vis_im.shape[1], vis_im.shape[0]), interpolation=cv2.INTER_NEAREST)

    # 최종 이미지 생성: 피부와 코 부분은 원본 색상, 나머지 부분은 검은색
    result = np.zeros_like(vis_im)  # 결과 이미지를 검은색으로 초기화
    mask_skin_and_nose = np.isin(vis_parsing_anno, [1, 10])  # 피부와 코의 마스크 생성

    # 잡음 제거를 위한 모폴로지 연산 적용 (침식 후 팽창: 열기 연산)
    kernel = np.ones((5, 5), np.uint8)
    mask_skin_and_nose = mask_skin_and_nose.astype(np.uint8) * 255
    mask_skin_and_nose = cv2.morphologyEx(mask_skin_and_nose, cv2.MORPH_OPEN, kernel)

    # 마스크를 다시 이진화하여 논리 연산에 사용할 수 있도록 변환
    mask_skin_and_nose = mask_skin_and_nose > 0

    # 피부와 코 부분은 원본 이미지 색상으로 채움
    result[mask_skin_and_nose] = vis_im[mask_skin_and_nose]

    # 결과 저장
    if save_im:
        cv2.imwrite(save_path[:-4] + '.png', vis_parsing_anno_color) 
        cv2.imwrite(save_path, cv2.cvtColor(result, cv2.COLOR_BGR2RGB), [int(cv2.IMWRITE_JPEG_QUALITY), 100])




In [82]:
def evaluate(respth, dspth, cp='model_final_diss.pth'):

    if not os.path.exists(respth):
        os.makedirs(respth)

    n_classes = 19
    net = BiSeNet(n_classes=n_classes)
    net.cuda()
    save_pth = osp.join('', cp)
    net.load_state_dict(torch.load(save_pth))
    net.eval()

    to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    with torch.no_grad():
        for image_path in os.listdir(dspth):
            img = Image.open(osp.join(dspth, image_path))
            image = img.resize((512, 512), Image.BILINEAR)
            img = to_tensor(image)
            img = torch.unsqueeze(img, 0)
            img = img.cuda()
            out = net(img)[0]
            parsing = out.squeeze(0).cpu().numpy().argmax(0)
            # print(parsing)
            print(np.unique(parsing))
                # 마스크 파일(npy) 저장
            npy_save_path = osp.join(respth, image_path.replace('.png', '.npy'))
            np.save(npy_save_path, parsing)

            # vis_parsing_maps(image, parsing, stride=1, save_im=True, save_path=osp.join(respth, image_path))


In [83]:
if __name__ == "__main__":
    base_input_folder = './data/images1024x1024'
    base_output_folder = './data/etcs/train_label'
    checkpoint_path = '79999_iter.pth'

    # 00000부터 50000까지 1000 단위로 폴더를 반복
    for i in range(51000, 70000, 1000):
        folder_name = str(i).zfill(5)  # '00000', '01000', ..., '50000'
        dspth = osp.join(base_input_folder, folder_name)
        respth = osp.join(base_output_folder, folder_name)

        if os.path.exists(dspth):
            print(f"Processing folder: {folder_name}")
            evaluate(respth=respth, dspth=dspth, cp=checkpoint_path)
        else:
            print(f"Skipping folder: {folder_name} (does not exist)")

Processing folder: 51000
[ 0  1  4  5 10 12 13 14 16 17]


/tmp/ipykernel_19479/3026376430.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(save_pth))


[ 0  1  2  3  4  5  6  7  8 10 12 13 14 16 17 18]
[ 0  1  2  3  6  7  8 10 12 13 14 16 17]
[ 0  1  2  3  8 10 11 12 13 14 16 17]
[ 0  1  2  3  4  5  7  8  9 10 12 13 14 16 17]
[ 0  1  2  3  4  5  8 10 11 12 13 14 16 17]
[ 0  1  2  3  4  5  7 10 11 12 13 14 16 17]
[ 0  1  2  3  4  5 10 12 13 14 16 17 18]
[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 16 17 18]
[ 0  1  2  3  4  5  7 10 11 12 13 14 17]
[ 0  1  2  3  4  5  7 10 11 12 13 14 16 17]
[ 0  1  2  4  6  7  8  9 10 11 12 13 14 16 17]
[ 0  1  2  3  6  7  8 10 12 13 14 16 17]
[ 0  1  2  3  6  7  8 10 11 12 13 14 16 17]
[ 0  1  2  3  6 10 11 12 13 14 16 17]
[ 0  1  2  3  4  5  7  8 10 11 12 13 14 16 17 18]
[ 0  1  2  3  6  7  8  9 10 12 13 14 15 16 17]
[ 0  1  2  3  4  5  7  8 10 11 12 13 16 17]
[ 0  1  2  3  4  5  7  9 10 12 13 14 16 17]
[ 0  1  2  3  6 10 11 12 13 14 16 17]
[ 0  1  2  3  6  7  8 10 12 13 14 16 17]
[ 0  1  2  3  4  5  7  8 10 12 13 14 17]
[ 0  1  2  3  4  5  8 10 11 12 13 14 16 17]
[ 0  1  2  3  4  5  7  8  9 10 11 12 1